In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import tensorflow
from tensorflow import keras
# from tensorflowkeras.utils.vis_utils import plot_model
import random
from sklearn.model_selection import train_test_split
from scipy import ndimage
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image

from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization,Conv2D,MaxPooling2D,Activation,Dropout,Lambda,Dense,Flatten,Input

import tensorflow as tf

import time
import psutil
import csv
import gc
import pickle

2024-05-22 21:02:07.597628: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-22 21:02:07.600067: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-22 21:02:07.628131: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-22 21:02:08.106047: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
class custom_MTL():
    def __init__(self) -> None:
        
        self.layer_list=[]
        self.weight_set=False
        self.partition_done = False
        x_shape, y_angle_shape, y_cifar10_shape=(50000, 32, 32, 3), (50000, 4), (50000, 10)
        angle_classifier_no=4
        cifar10_classifier_no=10
        inputs = keras.Input((32, 32, 3))
        conv_base = self.get_convbase(inputs)
        angle_classifier = self.get_classifier(conv_base, angle_classifier_no, "angle",count=15)
        cifar10_classifier = self.get_classifier(conv_base, cifar10_classifier_no, "cifar10",count=22)
        self.model = Model(
            inputs=inputs, 
            outputs=[cifar10_classifier, angle_classifier]
        )
        tf.keras.utils.plot_model(self.model, to_file='final_mtl_model.png', show_shapes=True,show_layer_names=True)
    
   
    def get_convbase(self,inputs):
    
    # reg = keras.regularizers.l2(1e-4)
    
    # initializer = keras.initializers.HeNormal()


        x = Conv2D(16, (3, 3), padding="same",name='1_conv2D_1')(inputs)
        x = Activation("relu",name='2_activation_1')(x)
        x = BatchNormalization(axis=-1,name='3_batch_norm_1')(x)
        x = MaxPooling2D(pool_size=(3, 3),name='4_maxPool_1')(x)
        x = Dropout(0.25,name='5_dropout_1')(x)
        
        x = Conv2D(32, (3, 3), padding="same",name='6_conv2D_2')(x)
        x = Activation("relu",name='7_activation_2')(x)
        x = BatchNormalization(axis=-1,name='8_batch_norm_2')(x)
        x = MaxPooling2D(pool_size=(2, 2),name='9_maxPool_2')(x)
        x = Dropout(0.25,name='10_dropout_2')(x)
        
        x = Conv2D(32, (3, 3), padding="same",name='11_conv2D_3')(x)
        x = Activation("relu",name='12_activation_3')(x)
        x = BatchNormalization(axis=-1,name='13_batch_norm_3')(x)
        x = MaxPooling2D(pool_size=(2, 2),name='14_maxPool_3')(x)
        x = Dropout(0.25,name='15_dropout_3')(x)
        
        return x
    
    def get_classifier(self,x, class_no, name,count):
    
        x = Flatten(name=f'{count+1}_layer')(x)
        x = Dense(128,name=f'{count+2}_layer')(x)
        x = Activation("relu",name=f'{count+3}_layer')(x)
        x = BatchNormalization(name=f'{count+4}_layer')(x)
        x = Dropout(0.5,name=f'{count+5}_layer')(x)
        x = Dense(class_no,name=f'{count+6}_layer')(x)
        x = Activation("softmax", name=name)(x)

        return x

    def load_weights(self):
        self.model.load_weights('./custom_mtl_model.h5')
        self.weight_set=True
        return
    def load_input(self):
        self.input_data = np.random.rand(1, 32, 32, 3).astype(np.float32)
        self.input_loaded=True
        return self.input_data
        
    def save_pickeled_layers(self):
        if not self.weight_set:
            self.loadWeights()

        
        if not self.partition_done:
            self.make_partition()
        save_dir='./../pickle_layers'
        for i in range(len(self.layer_list)):
            fname=f'./{save_dir}/custum_mtl_layer_{i}.pkl'
            layer_weights_and_config = {
                'weights': self.layer_list[i].get_weights(),
                'config': tf.keras.layers.serialize(self.layer_list[i])}
            with open(fname, 'wb') as f:
                pickle.dump(layer_weights_and_config, f)
             
    def load_layer(self, layer_id):
        self.dir_path='pickled_layers'
        fname=f'./{self.dir_path}/squeeznet_layer_{layer_id}.pkl'
        with open(fname, 'rb') as f:
            layer_config = pickle.load(f)
            
        self.layer = tf.keras.layers.deserialize(config= layer_config['config'])
            
        # self.layer_list.append(layer)
        self.layer.set_weights(layer_config['weights'])
        return self.layer   
                
    def make_partition(self):
        self.layer_list=[]
        self.NO_OF_LAYERS= len(self.model.layers)
        
        for i in range(self.NO_OF_LAYERS):
            self.temp_layer=self.model.layers[i]
            self.layer_list.append(self.temp_layer)
            
        self.partition_done = True
        print('\_______Partitioning Done')
        
    def execute_predict(self,input_data):
        st1=time.perf_counter()
        out=self.model.predict(input_data)
        et1=time.perf_counter()
        el1=et1-st1
        print(f'Elapsed Time: {el1}')
        return out
    
    def print_lays(self):
        idx=0
        for lay in self.model.layers:
            self.layer_list.append(lay)
            print(f'{idx} : {lay.input}')
            idx=idx+1
        print(f'Total number of Layers : {len(self.layer_list)}')
        
        
    def execute_lbl(self,input_data):
        st2=time.perf_counter()
        out=buffer=input_data
        for idx in range(1,len(self.model.layers)):
            
            if idx <= 15:
                print(f'Executing index {idx} : {self.model.layers[idx]}')
                out=buffer=self.model.layers[idx](out)
            elif idx in [16,18,20,22,24,26,28]:
                print(f'Executing index {idx} : {self.model.layers[idx]}')
                out=self.model.layers[idx](out)
            elif idx in [17,19,21,23,25,27,29]:
                print(f'Executing index {idx} : {self.model.layers[idx]}')
                buffer=self.model.layers[idx](buffer)
        et2=time.perf_counter()
        el2=et2-st2
        print(f'Elapsed Time: {el2}')
        return out,buffer
    
    def execute_lbl_serial(self,input_data):
        st2=time.perf_counter()
        out=buffer=input_data
        for idx in range(1,len(self.model.layers)):
            
            curr_lay=self.model.layers[idx]
            if idx <= 15:
                print(f'Executing index {idx} : {self.model.layers[idx]}')
                out=buffer=curr_lay(out)
            elif idx in [16,18,20,22,24,26,28]:
                print(f'Executing index {idx} : {self.model.layers[idx]}')
                out=curr_lay(out)
            elif idx in [17,19,21,23,25,27,29]:
                print(f'Executing index {idx} : {self.model.layers[idx]}')
                buffer=curr_lay(buffer)
        et2=time.perf_counter()
        el2=et2-st2
        print(f'Elapsed Time: {el2}')
        return out,buffer
    
    def gte_input_list(self,input_data):
        self.input_list=[]
        st2=time.perf_counter()
        out=buffer=input_data
        self.input_list.append(input_data)
        for idx in range(1,len(self.model.layers)):
            # print(f'Executing index {idx} :  {self.model.layers[idx]}')
            if idx <= 15:
                self.input_list.append(out)
                out=buffer=self.model.layers[idx](out)
            elif idx in [16,18,20,22,24,26,28]:
                self.input_list.append(out)
                out=self.model.layers[idx](out)
            elif idx in [17,19,21,23,25,27,29]:
                self.input_list.append(buffer)
                buffer=self.model.layers[idx](buffer)
        et2=time.perf_counter()
        el2=et2-st2
        print(f'Elapsed Time: {el2}')
        return self.input_list
    
    def execute_on_core(self,layer_id,input_data):
        # dummy_data=dummy_data
        # print(self.layer_list[layer_id].name)
        self.temp_out=self.layer_list[layer_id](input_data)
        
        return self.temp_out
        
    


In [3]:
NO_OF_LAYERS=30
NO_OF_CPU=24

In [4]:
image = np.random.rand(1,32,32,3)

In [5]:
obj=custom_MTL()
obj.load_weights()
obj.model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 32, 32, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ 1_conv2D_1 (Conv2D) │ (None, 32, 32,    │        448 │ input_layer[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ 2_activation_1      │ (None, 32, 32,    │          0 │ 1_conv2D_1[0][0]  │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ 3_batch_norm_1      │ (None, 32, 32,    │         64 │ 2_activation_1[0… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ 4_maxPool_1         │ (None, 10, 10,    │          0 │ 3_batch_norm_1[0… │
│ (MaxPooling2D)      │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ 5_dropout_1         │ (None, 10, 10,    │          0 │ 4_maxPool_1[0][0] │
│ (Dropout)           │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ 6_conv2D_2 (Conv2D) │ (None, 10, 10,    │      4,640 │ 5_dropout_1[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ 7_activation_2      │ (None, 10, 10,    │          0 │ 6_conv2D_2[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ 8_batch_norm_2      │ (None, 10, 10,    │        128 │ 7_activation_2[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ 9_maxPool_2         │ (None, 5, 5, 32)  │          0 │ 8_batch_norm_2[0… │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ 10_dropout_2        │ (None, 5, 5, 32)  │          0 │ 9_maxPool_2[0][0] │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ 11_conv2D_3         │ (None, 5, 5, 32)  │      9,248 │ 10_dropout_2[0][… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ 12_activation_3     │ (None, 5, 5, 32)  │          0 │ 11_conv2D_3[0][0] │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ 13_batch_norm_3     │ (None, 5, 5, 32)  │        128 │ 12_activation_3[… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ 14_maxPool_3        │ (None, 2, 2, 32)  │          0 │ 13_batch_norm_3[… │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ 15_dropout_3        │ (None, 2, 2, 32)  │          0 │ 14_maxPool_3[0][… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ 23_layer (Flatten)  │ (None, 128)       │          0 │ 15_dropout_3[0][

 Total params: 50,510 (197.30 KB)

 Trainable params: 49,838 (194.68 KB)

 Non-trainable params: 672 (2.62 KB)

In [ ]:

# out=obj.model.predict(image)
obj.print_lays()
obj.execute_predict(image)

In [10]:
out2=obj.execute_lbl_serial(image)

Executing index 1 : <Conv2D name=1_conv2D_1, built=True>
Executing index 2 : <Activation name=2_activation_1, built=True>
Executing index 3 : <BatchNormalization name=3_batch_norm_1, built=True>
Executing index 4 : <MaxPooling2D name=4_maxPool_1, built=True>
Executing index 5 : <Dropout name=5_dropout_1, built=True>
Executing index 6 : <Conv2D name=6_conv2D_2, built=True>
Executing index 7 : <Activation name=7_activation_2, built=True>
Executing index 8 : <BatchNormalization name=8_batch_norm_2, built=True>
Executing index 9 : <MaxPooling2D name=9_maxPool_2, built=True>
Executing index 10 : <Dropout name=10_dropout_2, built=True>
Executing index 11 : <Conv2D name=11_conv2D_3, built=True>
Executing index 12 : <Activation name=12_activation_3, built=True>
Executing index 13 : <BatchNormalization name=13_batch_norm_3, built=True>
Executing index 14 : <MaxPooling2D name=14_maxPool_3, built=True>
Executing index 15 : <Dropout name=15_dropout_3, built=True>
Executing index 16 : <Flatten name

In [11]:
out2

(<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
 array([[0.08035607, 0.07880184, 0.07140041, 0.10866018, 0.10823656,
         0.10098613, 0.165687  , 0.08122392, 0.09683332, 0.10781459]],
       dtype=float32)>,
 <tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[0.2735577 , 0.21228214, 0.2644962 , 0.24966398]], dtype=float32)>)